In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

## Configuraciones Iniciales
# Diccionario para convertir el número de mes a su nombre en español
meses = {
    1: "Enero", 2: "Febrero", 3: "Marzo", 4: "Abril", 5: "Mayo", 6: "Junio",
    7: "Julio", 8: "Agosto", 9: "Septiembre", 10: "Octubre", 11: "Noviembre", 12: "Diciembre"
}


## Leer df de variaciones mensuales IPC
df_ipc = pd.read_csv(r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\01  - Precio Vivienda\Distritos\IPC.csv", sep=";")

#Poner mes y año en columnas distintas
df_ipc['Año'] = df_ipc['Periodo'].apply(lambda x: x[:4]).astype(int)
df_ipc['Mes'] = df_ipc['Periodo'].apply(lambda x: x[5:]).astype(int).map(meses) 

#Solo datos entre 2011 y 2023
df_ipc = df_ipc[(df_ipc['Año'] >= 2011) & (df_ipc['Año'] <= 2023)]

#Ver columna y datos combinados
#print(df_ipc.columns)
#print(df_ipc.head())

## Precios Vivienda (€/m2)
# Cargar el SHP y el CSV Distrito
df_geografiaDistrito = gpd.read_file(r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Shapefiles\da_cartografiaDistritoMunicipal-25830\da_cartografiaDistritoMunicipalPolygon.shp")
df_preciosViviendaDistrito = pd.read_csv(r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\01  - Precio Vivienda\Distritos\Precios Vivienda Malaga.csv", sep=";")

# Ver las columnas
#print(df_geografiaDistrito.columns)
#print(df_preciosViviendaDistrito.columns)

# Opcional: inspeccionar los primeros registros
#print(df_geografiaDistrito.head())
#print(df_preciosViviendaDistrito.head())

# Combinar las capas por una columna común 
df = df_geografiaDistrito.merge(df_preciosViviendaDistrito, on="NUMERO").copy()

#Ver columna y datos combinados
#print(df.columns)
#print(df.head())

# Supongamos que df es tu DataFrame original
# Transformamos a formato largo usando pd.melt
df_long = pd.melt(df, id_vars=['ID_DISMUNI', 'NUMERO', 'NOMBRE', 'geometry', 'Distrito'],
                  var_name='Fecha', value_name='Precio €/m2')

#Limpieza de Precio €/m2
df_long[r'Precio €/m2'] = df_long[r'Precio €/m2'].replace('nd', np.nan)
df_long = df_long.dropna()
#Poner Precio €/m2 como float
df_long[r'Precio €/m2'] = df_long[r'Precio €/m2'].astype(float)

# Extraemos Mes y Año de la columna 'Fecha'
df_long['Mes'] = df_long['Fecha'].apply(lambda x: x.split(' ')[0])  # Extrae el mes
df_long['Año'] = df_long['Fecha'].apply(lambda x: x.split(' ')[1]).astype(int)  # Extrae el año

# Finalmente, puedes eliminar la columna 'Fecha' si ya no es necesaria
df_long = df_long.drop(columns=['Fecha'])

def ajustar_precio_ipc(row):
    precio_ajustado = row[r'Precio €/m2']
    for year in range(row['Año']+1, 2024):  # Ajuste desde el año 'year' hasta 2023
        inflacion = df_ipc[(df_ipc['Año'] == year) & (df_ipc['Mes'] == row['Mes'])]['Total'].values
        if len(inflacion) > 0:
            precio_ajustado *= (1+inflacion[0]/100)  # Ajustar por la inflación de ese año
    return precio_ajustado

df_long['Precio €/m2 Inf'] = df_long.apply(ajustar_precio_ipc, axis=1)

#Hacemos dos dataframes uno para 2011 y otro para 2021 teniendo en cuenta un periodo de 3 años (no disponemos de 5 años desde 2021 y 2024 no ha terminado) 
#2011-2013
df_long_2011 = df_long[(df_long['Año'] >= 2011) & (df_long['Año'] <= 2013)].copy()
#print(df_long_2011.head())
#2021-2023
df_long_2021 = df_long[(df_long['Año'] >= 2021) & (df_long['Año'] <= 2023)].copy()
#print(df_long_2021.head())

# Agrupación por area
df_long_2011_median = df_long_2011.groupby('NOMBRE').agg(
    median_price_inf=('Precio €/m2 Inf', 'median'),  # Mediana de Precio €/m2 Inf
    count=('Precio €/m2 Inf', 'size')  # Recuento de valores en Precio €/m2 Inf
).reset_index()
#print(df_long_2011_median)

df_long_2021_median = df_long_2021.groupby('NOMBRE').agg(
    median_price_inf=('Precio €/m2 Inf', 'median'),  # Mediana de Precio €/m2 Inf
    count=('Precio €/m2 Inf', 'size')  # Recuento de valores en Precio €/m2 Inf
).reset_index()
#print(df_long_2021_median)

#Incluir campos de geolocalizacion
#2011
df_long_2011_median_mapped = pd.merge(df_long_2011_median, df_geografiaDistrito, on='NOMBRE')
print(df_long_2011_median_mapped)
#2021
df_long_2021_median_mapped = pd.merge(df_long_2021_median, df_geografiaDistrito, on='NOMBRE')
print(df_long_2021_median_mapped)

# Export
#df_long_2011_median_mapped.to_csv(r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\01  - Precio Vivienda\Distritos\2011_Distritos_PrecioVivienda.csv",index=False,sep=";")
#df_long_2021_median_mapped.to_csv(r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\01  - Precio Vivienda\Distritos\2021_Distritos_PrecioVivienda.csv",index=False,sep=";")

#Export 2 (Menos campos)
columnas_exportacion_v2= ['NOMBRE', 'NUMERO', 'median_price_inf']
df_long_2011_median_mapped_v2 = df_long_2011_median_mapped[columnas_exportacion_v2]
df_long_2021_median_mapped_v2 = df_long_2021_median_mapped[columnas_exportacion_v2]
df_long_2011_median_mapped_v2.to_csv(r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\01  - Precio Vivienda\Distritos\2011_Distritos_PrecioVivienda_V2.csv",index=False,sep=";")
df_long_2021_median_mapped_v2.to_csv(r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\01  - Precio Vivienda\Distritos\2021_Distritos_PrecioVivienda_V2.csv",index=False,sep=";")


                 NOMBRE  median_price_inf  count  ID_DISMUNI  NUMERO  \
0     BAILEN-MIRAFLORES       1659.015611     36        95.0     4.0   
1           CAMPANILLAS       1728.567912     36       135.0     9.0   
2    CARRETERA DE CADIZ       2017.285959     36        92.0     7.0   
3                CENTRO       2361.290759     36       110.0     1.0   
4             CHURRIANA       2089.150808     36        91.0     8.0   
5         CIUDAD JARDIN       1581.765436     36        99.0     3.0   
6   CRUZ DE HUMILLADERO       1778.834742     36        94.0     6.0   
7                  ESTE       2846.996138     36       100.0     2.0   
8    PUERTO DE LA TORRE       2018.081334     36       133.0    10.0   
9  TEATINOS-UNIVERSIDAD       2257.217173     36        93.0    11.0   

                                            geometry  
0  POLYGON Z ((370858.973 4067157.454 0.000, 3708...  
1  POLYGON Z ((365919.285 4073105.832 0.000, 3659...  
2  POLYGON Z ((372234.992 4063847.501 0.00